#connect to mongo db

In [23]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["StorageMonitoring"]

# List all collections
collections = db.list_collection_names()

# Function to fetch and display data from each collection
def fetch_and_display_data():
    for collection_name in collections:
        collection = db[collection_name]  # Access collection
        data = list(collection.find({}, {"_id": 0}))  # Fetch all data, exclude _id

        if data:  # Check if collection has data
            df = pd.DataFrame(data)
            print(f"\n📌 Data from Collection: **{collection_name}**")
            display(df)  # Show as DataFrame
        else:
            print(f"\n⚠️ Collection '{collection_name}' is empty!")

# Run the function to display data
fetch_and_display_data()


📌 Data from Collection: **info**


,Timestamp,Directory,Files Added (GB),Files Deleted (GB),Files Modified (GB),Current Space (GB)
0,11/16/2024 15:13,/info,2.879527,1.127070,0.097263,61.752457
1,11/16/2024 15:28,/info,0.500424,1.187658,0.451119,61.065223
2,11/16/2024 15:43,/info,0.996160,2.352707,0.043864,59.708675
3,11/16/2024 15:58,/info,2.039046,0.538798,0.417166,61.208923
4,11/16/2024 16:13,/info,0.520040,1.446120,0.130248,60.282843
...,...,...,...,...,...,...
11516,3/16/2025 14:13,/info,0.782992,1.789093,0.356441,1498.993899
11517,3/16/2025 14:28,/info,0.671854,0.891764,0.306689,1498.773989
11518,3/16/2025 14:43,/info,0.226015,0.247917,0.165864,1498.752087
11519,3/16/2025 14:58,/info,1.465421,0.626116,0.335436,1499.591393



📌 Data from Collection: **scratch**


,Timestamp,Directory,Files Added (GB),Files Deleted (GB),Files Modified (GB),Current Space (GB)
0,11/16/2024 15:13,/scratch,1.439764,0.563535,0.048631,30.876229
1,11/16/2024 15:28,/scratch,0.250219,0.593846,0.225566,30.533495
2,11/16/2024 15:43,/scratch,0.498109,1.176422,0.021933,29.856065
3,11/16/2024 15:58,/scratch,1.019611,0.269422,0.208601,30.607118
4,11/16/2024 16:13,/scratch,0.260050,0.723144,0.065131,30.144910
...,...,...,...,...,...,...
11516,3/16/2025 14:13,/scratch,0.521949,1.192625,0.237607,999.242518
11517,3/16/2025 14:28,/scratch,0.447874,0.594471,0.204446,999.117608
11518,3/16/2025 14:43,/scratch,0.150670,0.165271,0.110571,999.124692
11519,3/16/2025 14:58,/scratch,0.976926,0.417402,0.223619,999.705900



📌 Data from Collection: **projects**


,Timestamp,Directory,Files Added (GB),Files Deleted (GB),Files Modified (GB),Current Space (GB)
0,11/16/2024 15:13,/projects,1.919685,0.751380,0.064842,41.168305
1,11/16/2024 15:28,/projects,0.333616,0.791772,0.300746,40.710149
2,11/16/2024 15:43,/projects,0.664107,1.568472,0.029243,39.805784
3,11/16/2024 15:58,/projects,1.359364,0.359199,0.278111,40.805949
4,11/16/2024 16:13,/projects,0.346693,0.964080,0.086832,40.188562
...,...,...,...,...,...,...
11516,3/16/2025 14:13,/projects,0.521995,1.192729,0.237628,999.329266
11517,3/16/2025 14:28,/projects,0.447903,0.594509,0.204459,999.182659
11518,3/16/2025 14:43,/projects,0.150677,0.165278,0.110576,999.168058
11519,3/16/2025 14:58,/projects,0.976947,0.417411,0.223624,999.727595



📌 Data from Collection: **customers**


,Timestamp,Directory,Files Added (GB),Files Deleted (GB),Files Modified (GB),Current Space (GB)
0,11/16/2024 15:13,/customer,4.799212,1.878450,0.162105,102.920762
1,11/16/2024 15:28,/customer,0.834040,1.979430,0.751865,101.775372
2,11/16/2024 15:43,/customer,1.660266,3.921179,0.073107,99.514459
3,11/16/2024 15:58,/customer,3.398410,0.897997,0.695277,102.014872
4,11/16/2024 16:13,/customer,0.866733,2.410200,0.217080,100.471405
...,...,...,...,...,...,...
11516,3/16/2025 14:13,/customer,1.304986,2.981822,0.594069,2498.323165
11517,3/16/2025 14:28,/customer,1.119757,1.486273,0.511148,2497.956648
11518,3/16/2025 14:43,/customer,0.376692,0.413194,0.276439,2497.920146
11519,3/16/2025 14:58,/customer,2.442368,1.043526,0.559059,2499.318988


In [27]:
for directory in directories:
    collection = db[directory]
    data = list(collection.find({}, {"_id": 0, "Timestamp": 1, "Current Space (GB)": 1}))
    
    if data:
        df = pd.DataFrame(data)
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")  # Convert properly
        df = df.sort_values("Timestamp")
        print(f"\n📊 First 5 records from {directory}:")
        print(df.head())  # Show first few records
    else:
        print(f"\n⚠️ No data found for {directory}")



📊 First 5 records from customers:
            Timestamp  Current Space (GB)
0 2024-11-16 15:13:00          102.920762
1 2024-11-16 15:28:00          101.775372
2 2024-11-16 15:43:00           99.514459
3 2024-11-16 15:58:00          102.014872
4 2024-11-16 16:13:00          100.471405

📊 First 5 records from info:
            Timestamp  Current Space (GB)
0 2024-11-16 15:13:00           61.752457
1 2024-11-16 15:28:00           61.065223
2 2024-11-16 15:43:00           59.708675
3 2024-11-16 15:58:00           61.208923
4 2024-11-16 16:13:00           60.282843

📊 First 5 records from projects:
            Timestamp  Current Space (GB)
0 2024-11-16 15:13:00           41.168305
1 2024-11-16 15:28:00           40.710149
2 2024-11-16 15:43:00           39.805784
3 2024-11-16 15:58:00           40.805949
4 2024-11-16 16:13:00           40.188562

📊 First 5 records from scratch:
            Timestamp  Current Space (GB)
0 2024-11-16 15:13:00           30.876229
1 2024-11-16 15:28:00       

In [30]:
for directory in ["customers", "info", "projects", "scratch"]:
    collection = db[directory]
    sample_entry = collection.find_one({}, {"Timestamp": 1, "_id": 0})
    print(f"\n📊 Sample entry from {directory}: {sample_entry}")



📊 Sample entry from customers: {'Timestamp': '11/16/2024 15:13'}

📊 Sample entry from info: {'Timestamp': '11/16/2024 15:13'}

📊 Sample entry from projects: {'Timestamp': '11/16/2024 15:13'}

📊 Sample entry from scratch: {'Timestamp': '11/16/2024 15:13'}
